In [2]:
import itertools
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score, roc_curve
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from collections import Counter

import torch
import torch_sparse
import pyg_lib
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.nn import GATConv, Linear
import torch_geometric.transforms as T
from torch_geometric.loader import NeighborLoader
from torch_geometric.nn import GCNConv, GINConv

from typing import Callable, Optional

from torch_geometric.data import (
    Data,
    InMemoryDataset
)

In [3]:
path = 'HI-Small_Trans.csv'
df = pd.read_csv(path)

In [43]:
def df_label_encoder(df, columns):
        le = preprocessing.LabelEncoder()
        for i in columns:
            df[i] = le.fit_transform(df[i].astype(str))
        return df

def preprocess(df):
        df = df_label_encoder(df,['Payment Format', 'Payment Currency', 'Receiving Currency'])
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df['Timestamp'] = df['Timestamp'].apply(lambda x: x.value)
        df['Timestamp'] = (df['Timestamp']-df['Timestamp'].min())/(df['Timestamp'].max()-df['Timestamp'].min())

        df['Account'] = df['From Bank'].astype(str) + '_' + df['Account']
        df['Account.1'] = df['To Bank'].astype(str) + '_' + df['Account.1']
        df = df.sort_values(by=['Account'])
        receiving_df = df[['Account.1', 'Amount Received', 'Receiving Currency']]
        paying_df = df[['Account', 'Amount Paid', 'Payment Currency']]
        receiving_df = receiving_df.rename({'Account.1': 'Account'}, axis=1)
        currency_ls = sorted(df['Receiving Currency'].unique())

        return df, receiving_df, paying_df, currency_ls

In [44]:
df, receiving_df, paying_df, currency_ls = preprocess(df = df)
print(df.head())

         Timestamp  From Bank                Account  To Bank  \
4278714        0.0      10057  10057_10057_803A115E0    29467   
1541411        0.0      10057  10057_10057_803A115E0    29467   
1541410        0.0      10057  10057_10057_803A115E0    29467   
824238         0.0      10057  10057_10057_803A115E0    29467   
824237         0.0      10057  10057_10057_803A115E0    29467   

                     Account.1  Amount Received  Receiving Currency  \
4278714  29467_29467_803E020C0        787197.11                   5   
1541411  29467_29467_803E020C0        681262.19                   5   
1541410  29467_29467_803E020C0        787197.11                   5   
824238   29467_29467_803E020C0        681262.19                   5   
824237   29467_29467_803E020C0        787197.11                   5   

         Amount Paid  Payment Currency  Payment Format  Is Laundering  
4278714    787197.11                 5               3              0  
1541411    681262.19                 5

In [45]:
print(receiving_df.head())
print(paying_df.head())

                    Account111  Amount Received  Receiving Currency
4278714  29467_29467_803E020C0        787197.11                   5
1541411  29467_29467_803E020C0        681262.19                   5
1541410  29467_29467_803E020C0        787197.11                   5
824238   29467_29467_803E020C0        681262.19                   5
824237   29467_29467_803E020C0        787197.11                   5
                       Account  Amount Paid  Payment Currency
4278714  10057_10057_803A115E0    787197.11                 5
1541411  10057_10057_803A115E0    681262.19                 5
1541410  10057_10057_803A115E0    787197.11                 5
824238   10057_10057_803A115E0    681262.19                 5
824237   10057_10057_803A115E0    787197.11                 5


In [6]:
print(currency_ls)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


In [7]:
def get_all_account(df):
        ldf = df[['Account', 'From Bank']]
        rdf = df[['Account.1', 'To Bank']]
        suspicious = df[df['Is Laundering']==1]
        s1 = suspicious[['Account', 'Is Laundering']]
        s2 = suspicious[['Account.1', 'Is Laundering']]
        s2 = s2.rename({'Account.1': 'Account'}, axis=1)
        suspicious = pd.concat([s1, s2], join='outer')
        suspicious = suspicious.drop_duplicates()

        ldf = ldf.rename({'From Bank': 'Bank'}, axis=1)
        rdf = rdf.rename({'Account.1': 'Account', 'To Bank': 'Bank'}, axis=1)
        df = pd.concat([ldf, rdf], join='outer')
        df = df.drop_duplicates()

        df['Is Laundering'] = 0
        df.set_index('Account', inplace=True)
        df.update(suspicious.set_index('Account'))
        df = df.reset_index()
        return df

In [8]:
accounts = get_all_account(df)
print(accounts.head())

           Account   Bank  Is Laundering
0  10057_803A115E0  10057              0
1  10057_803AA8E90  10057              0
2  10057_803AAB430  10057              0
3  10057_803AACE20  10057              0
4  10057_803AB4F70  10057              0


In [9]:
def paid_currency_aggregate(currency_ls, paying_df, accounts):
        for i in currency_ls:
            temp = paying_df[paying_df['Payment Currency'] == i]
            accounts['avg paid '+str(i)] = temp['Amount Paid'].groupby(temp['Account']).transform('mean')
        return accounts

def received_currency_aggregate(currency_ls, receiving_df, accounts):
    for i in currency_ls:
        temp = receiving_df[receiving_df['Receiving Currency'] == i]
        accounts['avg received '+str(i)] = temp['Amount Received'].groupby(temp['Account']).transform('mean')
    accounts = accounts.fillna(0)
    return accounts

In [10]:
def get_node_attr(currency_ls, paying_df,receiving_df, accounts):
        node_df = paid_currency_aggregate(currency_ls, paying_df, accounts)
        node_df = received_currency_aggregate(currency_ls, receiving_df, node_df)
        node_label = torch.from_numpy(node_df['Is Laundering'].values).to(torch.float)
        node_df = node_df.drop(['Account', 'Is Laundering'], axis=1)
        node_df = df_label_encoder(node_df,['Bank'])
        return node_df, node_label

In [11]:
node_df, node_label = get_node_attr(currency_ls, paying_df,receiving_df, accounts)
print(node_df.head())

   Bank  avg paid 0  avg paid 1  avg paid 2  avg paid 3  avg paid 4  \
0     2         0.0         0.0         0.0         0.0         0.0   
1     2         0.0         0.0         0.0         0.0         0.0   
2     2         0.0         0.0         0.0         0.0         0.0   
3     2         0.0         0.0         0.0         0.0         0.0   
4     2         0.0         0.0         0.0         0.0         0.0   

   avg paid 5  avg paid 6  avg paid 7  avg paid 8  ...  avg received 5  \
0         0.0         0.0         0.0         0.0  ...             0.0   
1         0.0         0.0         0.0         0.0  ...             0.0   
2         0.0         0.0         0.0         0.0  ...             0.0   
3         0.0         0.0         0.0         0.0  ...             0.0   
4         0.0         0.0         0.0         0.0  ...             0.0   

   avg received 6  avg received 7  avg received 8  avg received 9  \
0             0.0             0.0             0.0          

In [12]:
def get_edge_df(accounts, df):
        accounts = accounts.reset_index(drop=True)
        accounts['ID'] = accounts.index
        mapping_dict = dict(zip(accounts['Account'], accounts['ID']))
        df['From'] = df['Account'].map(mapping_dict)
        df['To'] = df['Account.1'].map(mapping_dict)
        df = df.drop(['Account', 'Account.1', 'From Bank', 'To Bank'], axis=1)

        edge_index = torch.stack([torch.from_numpy(df['From'].values), torch.from_numpy(df['To'].values)], dim=0)

        df = df.drop(['Is Laundering', 'From', 'To'], axis=1)
        edge_attr = df
        return edge_attr, edge_index

In [13]:
edge_attr, edge_index = get_edge_df(accounts, df)
print(edge_attr.head())

         Timestamp  Amount Received  Receiving Currency  Amount Paid  \
4278714   0.456320        787197.11                  13    787197.11   
2798190   0.285018        787197.11                  13    787197.11   
2798191   0.284233        681262.19                  13    681262.19   
3918769   0.417079        681262.19                  13    681262.19   
213094    0.000746        146954.27                  13    146954.27   

         Payment Currency  Payment Format  
4278714                13               3  
2798190                13               3  
2798191                13               4  
3918769                13               4  
213094                 13               5  


In [14]:
print(edge_index)

tensor([[     0,      0,      0,  ..., 496997, 496997, 496998],
        [299458, 299458, 299458,  ..., 496997, 496997, 496998]])


In [4]:
class AMLtoGraph(InMemoryDataset):
    def __init__(self, root: str, edge_window_size: int = 10,
                 transform=None, pre_transform=None):
        self.edge_window_size = edge_window_size
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self) -> str:
        return 'HI-Small_Trans.csv'

    @property
    def processed_file_names(self) -> str:
        return 'data.pt'
    
    @property
    def num_nodes(self) -> int:
        return self._data.edge_index.max().item() + 1

    def df_label_encoder(self, df, columns):
        le = preprocessing.LabelEncoder()
        for i in columns:
            df[i] = le.fit_transform(df[i].astype(str))
        return df

    def preprocess(self, df):
        df = self.df_label_encoder(df,['Payment Format', 'Payment Currency', 'Receiving Currency'])
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df['Timestamp'] = df['Timestamp'].apply(lambda x: x.value)
        df['Timestamp'] = (df['Timestamp']-df['Timestamp'].min())/(df['Timestamp'].max()-df['Timestamp'].min())

        df['Account'] = df['From Bank'].astype(str) + '_' + df['Account']
        df['Account.1'] = df['To Bank'].astype(str) + '_' + df['Account.1']
        df = df.sort_values(by=['Account'])
        receiving_df = df[['Account.1', 'Amount Received', 'Receiving Currency']]
        paying_df = df[['Account', 'Amount Paid', 'Payment Currency']]
        receiving_df = receiving_df.rename({'Account.1': 'Account'}, axis=1)
        currency_ls = sorted(df['Receiving Currency'].unique())

        return df, receiving_df, paying_df, currency_ls

    def get_all_account(self, df):
        ldf = df[['Account', 'From Bank']]
        rdf = df[['Account.1', 'To Bank']]
        suspicious = df[df['Is Laundering']==1]
        s1 = suspicious[['Account', 'Is Laundering']]
        s2 = suspicious[['Account.1', 'Is Laundering']]
        s2 = s2.rename({'Account.1': 'Account'}, axis=1)
        suspicious = pd.concat([s1, s2], join='outer')
        suspicious = suspicious.drop_duplicates()

        ldf = ldf.rename({'From Bank': 'Bank'}, axis=1)
        rdf = rdf.rename({'Account.1': 'Account', 'To Bank': 'Bank'}, axis=1)
        df = pd.concat([ldf, rdf], join='outer')
        df = df.drop_duplicates()

        df['Is Laundering'] = 0
        df.set_index('Account', inplace=True)
        df.update(suspicious.set_index('Account'))
        df = df.reset_index()
        return df
    
    def paid_currency_aggregate(self, currency_ls, paying_df, accounts):
        for i in currency_ls:
            temp = paying_df[paying_df['Payment Currency'] == i]
            accounts['avg paid '+str(i)] = temp['Amount Paid'].groupby(temp['Account']).transform('mean')
        return accounts

    def received_currency_aggregate(self, currency_ls, receiving_df, accounts):
        for i in currency_ls:
            temp = receiving_df[receiving_df['Receiving Currency'] == i]
            accounts['avg received '+str(i)] = temp['Amount Received'].groupby(temp['Account']).transform('mean')
        accounts = accounts.fillna(0)
        return accounts

    def get_edge_df(self, accounts, df):
        accounts = accounts.reset_index(drop=True)
        accounts['ID'] = accounts.index
        mapping_dict = dict(zip(accounts['Account'], accounts['ID']))
        df['From'] = df['Account'].map(mapping_dict)
        df['To'] = df['Account.1'].map(mapping_dict)
        df = df.drop(['Account', 'Account.1', 'From Bank', 'To Bank'], axis=1)

        edge_index = torch.stack([torch.from_numpy(df['From'].values), torch.from_numpy(df['To'].values)], dim=0)

        df = df.drop(['Is Laundering', 'From', 'To'], axis=1)

        edge_attr = torch.from_numpy(df.values).to(torch.float)
        return edge_attr, edge_index

    def get_node_attr(self, currency_ls, paying_df,receiving_df, accounts):
        node_df = self.paid_currency_aggregate(currency_ls, paying_df, accounts)
        node_df = self.received_currency_aggregate(currency_ls, receiving_df, node_df)
        node_label = torch.from_numpy(node_df['Is Laundering'].values).to(torch.float)
        node_df = node_df.drop(['Account', 'Is Laundering'], axis=1)
        node_df = self.df_label_encoder(node_df,['Bank'])
        node_df = torch.from_numpy(node_df.values).to(torch.float)
        return node_df, node_label


    def process(self):
        #load CSV file from raw
        raw_csv_path = os.path.join(self.raw_dir, self.raw_file_names)
        df = pd.read_csv(raw_csv_path)  #CSV file
        df, receiving_df, paying_df, currency_ls = self.preprocess(df)

        #create the accounts and graph features
        accounts = self.get_all_account(df)
        node_attr, node_label = self.get_node_attr(currency_ls, paying_df, receiving_df, accounts)
        edge_attr, edge_index = self.get_edge_df(accounts, df)

        #create the graph data object
        data = Data(x=node_attr, edge_index=edge_index, y=node_label, edge_attr=edge_attr)

        #processed data
        torch.save(self.collate([data]), self.processed_paths[0]) 

In [5]:
#models GAT, GCN, GIN
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads):
        super().__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads, dropout=0.6)
        self.conv2 = GATConv(hidden_channels * heads, int(hidden_channels/4), heads=1, concat=False, dropout=0.6)
        self.lin = Linear(int(hidden_channels/4), out_channels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, edge_index, edge_attr):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index, edge_attr))
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv2(x, edge_index, edge_attr))
        x = self.lin(x)
        x = self.sigmoid(x)
        return x

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, int(hidden_channels / 4))
        self.lin = Linear(int(hidden_channels / 4), out_channels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, edge_index, edge_attr=None):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))  
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv2(x, edge_index))
        x = self.lin(x)
        x = self.sigmoid(x)
        return x

class GIN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()

        nn1 = torch.nn.Sequential(
            Linear(in_channels, hidden_channels),
            torch.nn.ReLU(),
            Linear(hidden_channels, hidden_channels)
        )
        self.conv1 = GINConv(nn1)

        nn2 = torch.nn.Sequential(
            Linear(hidden_channels, int(hidden_channels/4)),
            torch.nn.ReLU(),
            Linear(int(hidden_channels/4), int(hidden_channels/4))
        )
        self.conv2 = GINConv(nn2)

        self.lin = Linear(int(hidden_channels/4), out_channels)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x, edge_index, edge_attr=None):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.relu(self.conv1(x, edge_index))

        x = F.dropout(x, p=0.6, training=self.training)
        x = F.relu(self.conv2(x, edge_index))

        x = self.lin(x)
        x = self.sigmoid(x)
        return x


In [6]:
################ Data Preparation  ###################
dataset = AMLtoGraph('C://Users//User//Desktop//AML')
data = dataset[0]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#split data for training, validation, and testing
split = T.RandomNodeSplit(split='train_rest', num_val=0.2, num_test=0.2)
data = split(data)

#create data loaders
train_loader = NeighborLoader(
    data,
    num_neighbors=[30] * 2,
    batch_size=256,
    input_nodes=data.train_mask,
)

val_loader = NeighborLoader(
    data,
    num_neighbors=[30] * 2,
    batch_size=256,
    input_nodes=data.val_mask,
)

test_loader = NeighborLoader(
    data,
    num_neighbors=[30] * 2,
    batch_size=256,
    input_nodes=data.test_mask,
)


In [7]:
#training and hyperparameter tuning
def train_and_fine_tune_model(model_name, data, hidden_channels, learning_rate, dropout_rate, num_epochs=20, heads=8):
    model_class = {
        "GIN": GIN,
        "GCN": GCN,
        "GAT": GAT
    }.get(model_name)

    if model_name == "GAT":
        model = model_class(in_channels=data.num_features, hidden_channels=hidden_channels, out_channels=1, heads=heads).to(device)
    else:
        model = model_class(in_channels=data.num_features, hidden_channels=hidden_channels, out_channels=1).to(device)

    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    criterion = torch.nn.BCELoss()

    patience = 3
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch_data in train_loader:
            optimizer.zero_grad()
            batch_data.to(device)
            pred = model(batch_data.x, batch_data.edge_index, batch_data.edge_attr)
            ground_truth = batch_data.y
            loss = criterion(pred, ground_truth.unsqueeze(1))
            loss.backward()
            optimizer.step()
            total_loss += float(loss)

        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for test_data in val_loader:
                test_data.to(device)
                pred = model(test_data.x, test_data.edge_index, test_data.edge_attr)
                ground_truth = test_data.y
                loss = criterion(pred, ground_truth.unsqueeze(1))
                val_loss += float(loss)
                correct += (pred.round() == ground_truth.unsqueeze(1)).sum().item()
                total += len(ground_truth)

        val_loss /= len(val_loader)
        val_accuracy = correct / total

        print(f"Epoch {epoch}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping due to no improvement.")
            break

    return model, best_val_loss, val_accuracy


In [19]:
#hyperparameter ranges
learning_rates = [0.0001, 0.001, 0.01]
hidden_channels_list = [16, 32, 64]
dropout_rates = [0.3, 0.5, 0.6]
num_epochs = 20

#store the best parameters and accuracy for each model
best_params_per_model = {}

models_to_tune = ["GIN", "GCN", "GAT"]

for model_name in models_to_tune:
    best_accuracy = 0
    best_params = {}

    for lr, hidden_channels, dropout_rate in itertools.product(learning_rates, hidden_channels_list, dropout_rates):
        heads = 8 if model_name == "GAT" else 1
        print(f"\nTuning {model_name} with lr={lr}, hidden_channels={hidden_channels}, dropout={dropout_rate}")

        model, val_loss, val_accuracy = train_and_fine_tune_model(model_name, data, hidden_channels, lr, dropout_rate, num_epochs, heads)

        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_params = {
                'learning_rate': lr,
                'hidden_channels': hidden_channels,
                'dropout_rate': dropout_rate,
                'heads': heads
            }

    best_params_per_model[model_name] = best_params
    print(f"Best parameters for {model_name}: {best_params}")
    print(f"Best validation accuracy for {model_name}: {best_accuracy}")



Tuning GIN with lr=0.0001, hidden_channels=16, dropout=0.3
Epoch 0, Validation Loss: 2.5738262338910447, Validation Accuracy: 0.9544457355927009
Epoch 1, Validation Loss: 1.1646821722203389, Validation Accuracy: 0.968798688196321
Epoch 2, Validation Loss: 0.619325004351642, Validation Accuracy: 0.9747201201784281
Epoch 3, Validation Loss: 0.6011371744181915, Validation Accuracy: 0.974734830679937
Epoch 4, Validation Loss: 0.6188000538213081, Validation Accuracy: 0.974134583946379
Epoch 5, Validation Loss: 0.5682596318775016, Validation Accuracy: 0.9745104650390555
Epoch 6, Validation Loss: 0.538902826108057, Validation Accuracy: 0.9746887500447015
Epoch 7, Validation Loss: 0.5204901406841893, Validation Accuracy: 0.9746761391649997
Epoch 8, Validation Loss: 0.5037702187415092, Validation Accuracy: 0.974798356181188
Epoch 9, Validation Loss: 0.4881391523789531, Validation Accuracy: 0.9747038178000521
Epoch 10, Validation Loss: 0.4730992476194727, Validation Accuracy: 0.9748097062579821

In [21]:
#evaluate the models with the best parameters
def evaluate_best_model(model_name, data, best_params):
    model_class = {
        "GIN": GIN,
        "GCN": GCN,
        "GAT": GAT
    }.get(model_name)

    if model_name == "GAT":
        model = model_class(in_channels=data.num_features, hidden_channels=best_params['hidden_channels'], out_channels=1, heads=best_params['heads']).to(device)
    else:
        model = model_class(in_channels=data.num_features, hidden_channels=best_params['hidden_channels'], out_channels=1).to(device)

    optimizer = torch.optim.SGD(model.parameters(), lr=best_params['learning_rate'])
    criterion = torch.nn.BCELoss()

    model.train()
    for epoch in range(num_epochs):
        for batch_data in train_loader:
            optimizer.zero_grad()
            batch_data.to(device)
            pred = model(batch_data.x, batch_data.edge_index, batch_data.edge_attr)
            ground_truth = batch_data.y
            loss = criterion(pred, ground_truth.unsqueeze(1))
            loss.backward()
            optimizer.step()

    #evaluation
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for test_data in test_loader:
            test_data.to(device)
            pred = model(test_data.x, test_data.edge_index, test_data.edge_attr)
            ground_truth = test_data.y

            #threshold predictions to convert probabilities into binary class labels
            pred_labels = (pred > 0.5).float()

            loss = criterion(pred, ground_truth.unsqueeze(1))
            test_loss += float(loss)

            correct += (pred_labels == ground_truth.unsqueeze(1)).sum().item()
            total += len(ground_truth)

            all_preds.extend(pred_labels.cpu().numpy())
            all_labels.extend(ground_truth.cpu().numpy())

    test_accuracy = correct / total
    test_loss /= len(test_loader)

    #classification metrics
    f1 = f1_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    precision = precision_score(all_labels, all_preds, average='binary')

    print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")
    print(f"Precision: {precision:.4f}, F1 Score: {f1:.4f}, Recall: {recall:.4f}")

for model_name in models_to_tune:
    print(f"\nEvaluating {model_name} model with the best hyperparameters...")
    best_params = best_params_per_model[model_name]
    evaluate_best_model(model_name, data, best_params)


Evaluating GIN model with the best hyperparameters...
Test Loss: 0.23379696151310989, Test Accuracy: 0.9753608665386645
Precision: 0.0000, F1 Score: 0.0000, Recall: 0.0000

Evaluating GCN model with the best hyperparameters...
Test Loss: 2.066562446375047, Test Accuracy: 0.9754703992657182
Precision: 0.0000, F1 Score: 0.0000, Recall: 0.0000

Evaluating GAT model with the best hyperparameters...
Test Loss: 1.9184506607528489, Test Accuracy: 0.8969665271966527
Precision: 0.0304, F1 Score: 0.0469, Recall: 0.1027


In [35]:
######### SMOTE WITH BCE #################

In [27]:
#SMOTE  function (SMOTE only to training data)
def apply_smote_to_train_data(data):
    X_train = data.x[data.train_mask].cpu().numpy()  #features for training nodes
    y_train = data.y[data.train_mask].cpu().numpy()  #labels for training nodes

    #apply SMOTE to the training data
    smote = SMOTE(sampling_strategy='auto', random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

    #convert the resampled data back to tensors
    X_resampled_tensor = torch.tensor(X_resampled, dtype=torch.float).to(device)
    y_resampled_tensor = torch.tensor(y_resampled, dtype=torch.long).to(device)

    #update the training portion of the data with the resampled data
    new_data = Data(
        x=torch.cat([data.x, X_resampled_tensor], dim=0),
        edge_index=data.edge_index,
        y=torch.cat([data.y, y_resampled_tensor], dim=0),
        edge_attr=data.edge_attr,
        train_mask=torch.cat([data.train_mask, torch.ones(X_resampled_tensor.size(0), dtype=torch.bool).to(device)], dim=0),
        val_mask=data.val_mask,
        test_mask=data.test_mask
    )
    return new_data

#apply SMOTE to the training data
data_smote = apply_smote_to_train_data(data)

#new data loader for the SMOTE training data
train_loader_smote = NeighborLoader(
    data_smote,
    num_neighbors=[30] * 2,
    batch_size=256,
    input_nodes=data_smote.train_mask,  #SMOTE training data
)

In [47]:
#training and hyperparameter tuning with SMOTE
def train_and_fine_tune_model_with_smote(model_name, data_smote, hidden_channels, learning_rate, dropout_rate, num_epochs=20, heads=8):
    model_class = {
        "GIN": GIN,
        "GCN": GCN,
        "GAT": GAT
    }.get(model_name)

    if model_name == "GAT":
        model = model_class(in_channels=data_smote.num_features, hidden_channels=hidden_channels, out_channels=1, heads=heads).to(device)
    else:
        model = model_class(in_channels=data_smote.num_features, hidden_channels=hidden_channels, out_channels=1).to(device)

    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    criterion = torch.nn.BCELoss()

    patience = 3
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch_data in train_loader_smote:  #SMOTE train loader
            optimizer.zero_grad()
            batch_data.to(device)
            pred = model(batch_data.x, batch_data.edge_index, batch_data.edge_attr)
            ground_truth = batch_data.y
            loss = criterion(pred, ground_truth.unsqueeze(1))
            loss.backward()
            optimizer.step()
            total_loss += float(loss)

        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for test_data in val_loader:
                test_data.to(device)
                pred = model(test_data.x, test_data.edge_index, test_data.edge_attr)
                ground_truth = test_data.y
                loss = criterion(pred, ground_truth.unsqueeze(1))
                val_loss += float(loss)
                correct += (pred.round() == ground_truth.unsqueeze(1)).sum().item()
                total += len(ground_truth)

        val_loss /= len(val_loader)
        val_accuracy = correct / total

        print(f"Epoch {epoch}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping due to no improvement.")
            break

    return model, best_val_loss, val_accuracy

In [48]:
#hyperparameter
learning_rates = [0.0001, 0.001, 0.01]
hidden_channels_list = [16, 32, 64]
dropout_rates = [0.3, 0.5, 0.6]
num_epochs = 20

best_params_per_model = {}

models_to_tune = ["GIN", "GCN", "GAT"]

for model_name in models_to_tune:
    best_accuracy = 0
    best_params = {}

    for lr, hidden_channels, dropout_rate in itertools.product(learning_rates, hidden_channels_list, dropout_rates):
        heads = 8 if model_name == "GAT" else 1
        print(f"\nTuning {model_name} with SMOTE and lr={lr}, hidden_channels={hidden_channels}, dropout={dropout_rate}")

        model, val_loss, val_accuracy = train_and_fine_tune_model_with_smote(model_name, data_smote, hidden_channels, lr, dropout_rate, num_epochs, heads)

        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_params = {
                'learning_rate': lr,
                'hidden_channels': hidden_channels,
                'dropout_rate': dropout_rate,
                'heads': heads
            }

    best_params_per_model[model_name] = best_params
    print(f"Best parameters for {model_name}: {best_params}")
    print(f"Best validation accuracy for {model_name}: {best_accuracy}")


Tuning GIN with SMOTE and lr=0.0001, hidden_channels=16, dropout=0.3
Epoch 0, Validation Loss: 66.21786625272878, Validation Accuracy: 0.025309537430532852
Epoch 1, Validation Loss: 12.598151302515129, Validation Accuracy: 0.025213713438709925
Epoch 2, Validation Loss: 10.227170972611118, Validation Accuracy: 0.03878296871805357
Epoch 3, Validation Loss: 5.297480532608316, Validation Accuracy: 0.9103371108000122
Epoch 4, Validation Loss: 2.226015207045723, Validation Accuracy: 0.9523403124504929
Epoch 5, Validation Loss: 0.6373486455261855, Validation Accuracy: 0.9748223588899763
Epoch 6, Validation Loss: 0.6176545050540574, Validation Accuracy: 0.9746904295553268
Epoch 7, Validation Loss: 0.6015880319379991, Validation Accuracy: 0.9747787101965744
Epoch 8, Validation Loss: 0.5789770086113336, Validation Accuracy: 0.9747625689048283
Epoch 9, Validation Loss: 0.5651549336040582, Validation Accuracy: 0.9747709528616517
Epoch 10, Validation Loss: 0.5528884416182639, Validation Accuracy: 

In [51]:
def evaluate_best_model(model_name, data, best_params):
    model_class = {
        "GIN": GIN,
        "GCN": GCN,
        "GAT": GAT
    }.get(model_name)

    if model_name == "GAT":
        model = model_class(in_channels=data.num_features, hidden_channels=best_params['hidden_channels'], out_channels=1, heads=best_params['heads']).to(device)
    else:
        model = model_class(in_channels=data.num_features, hidden_channels=best_params['hidden_channels'], out_channels=1).to(device)

    optimizer = torch.optim.SGD(model.parameters(), lr=best_params['learning_rate'])
    criterion = torch.nn.BCELoss()

    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for test_data in test_loader: 
            test_data.to(device)
            pred = model(test_data.x, test_data.edge_index, test_data.edge_attr)
            ground_truth = test_data.y

            #threshold predictions to convert probabilities into binary class labels
            pred_labels = (pred > 0.5).float()

            loss = criterion(pred, ground_truth.unsqueeze(1))
            test_loss += float(loss)

            correct += (pred_labels == ground_truth.unsqueeze(1)).sum().item()
            total += len(ground_truth)

            all_preds.extend(pred_labels.cpu().numpy())
            all_labels.extend(ground_truth.cpu().numpy())

    test_accuracy = correct / total
    test_loss /= len(test_loader)

    f1 = f1_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    precision = precision_score(all_labels, all_preds, average='binary')

    print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")
    print(f"Precision: {precision:.4f}, F1 Score: {f1:.4f}, Recall: {recall:.4f}")

In [52]:
#evaluate the models with the best parameters trained with SMOTE
for model_name in models_to_tune:
    print(f"\nEvaluating {model_name} model with SMOTE and the best hyperparameters...")
    best_params = best_params_per_model[model_name]
    evaluate_best_model(model_name, data, best_params)



Evaluating GIN model with SMOTE and the best hyperparameters...
Test Loss: 14.01820130520974, Test Accuracy: 0.8109274325095832
Precision: 0.0213, F1 Score: 0.0373, Recall: 0.1486

Evaluating GCN model with SMOTE and the best hyperparameters...
Test Loss: 44.33119998115555, Test Accuracy: 0.5265838864203325
Precision: 0.0266, F1 Score: 0.0505, Recall: 0.5111

Evaluating GAT model with SMOTE and the best hyperparameters...
Test Loss: 63.673060179348326, Test Accuracy: 0.3084712060570525
Precision: 0.0246, F1 Score: 0.0475, Recall: 0.7009


In [ ]:
#Focal Loss

In [18]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=1, weight=None, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.weight = weight
        self.reduction = reduction

    def forward(self, input, target):
        #sigmoid activation and probabilities
        pt = torch.sigmoid(input)

        #target float for proper multiplication
        target = target.float()

        #compute the focal loss term
        loss = -1 * (1 - pt) ** self.gamma * (target * torch.log(pt + 1e-9) + (1 - target) * torch.log(1 - pt + 1e-9))

        #apply class weights if provided
        if self.weight is not None:
            #weights broadcast correctly across target dimensions
            if self.weight.dim() == 1:
                weight_tensor = self.weight[target.long()]  #weight for each class in target
            else:
                weight_tensor = self.weight
            loss = loss * weight_tensor

        #reduction (mean or sum)
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()

        return loss

In [19]:
#training and hyperparameter tuning with FocalLoss
def train_and_fine_tune_model_focal_loss(model_name, data, hidden_channels, learning_rate, dropout_rate, num_epochs=20, heads=8):
    model_class = {
        "GIN": GIN,
        "GCN": GCN,
        "GAT": GAT
    }.get(model_name)

    if model_name == "GAT":
        model = model_class(in_channels=data.num_features, hidden_channels=hidden_channels, out_channels=1, heads=heads).to(device)
    else:
        model = model_class(in_channels=data.num_features, hidden_channels=hidden_channels, out_channels=1).to(device)

    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    criterion = FocalLoss(gamma=1, weight=torch.tensor([1.0, 2.0]).to(device))

    patience = 3
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch_data in train_loader:
            optimizer.zero_grad()
            batch_data.to(device)
            pred = model(batch_data.x, batch_data.edge_index, batch_data.edge_attr)
            ground_truth = batch_data.y
            loss = criterion(pred, ground_truth.unsqueeze(1))
            loss.backward()
            optimizer.step()
            total_loss += float(loss)

        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for test_data in val_loader:
                test_data.to(device)
                pred = model(test_data.x, test_data.edge_index, test_data.edge_attr)
                ground_truth = test_data.y
                loss = criterion(pred, ground_truth.unsqueeze(1))
                val_loss += float(loss)
                correct += (pred.round() == ground_truth.unsqueeze(1)).sum().item()
                total += len(ground_truth)

        val_loss /= len(val_loader)
        val_accuracy = correct / total

        print(f"Epoch {epoch}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping due to no improvement.")
            break

    return model, best_val_loss, val_accuracy


In [20]:
#hyperparameter
learning_rates = [0.0001, 0.001, 0.01]
hidden_channels_list = [16, 32, 64]
dropout_rates = [0.3, 0.5, 0.6]
num_epochs = 20

best_params_per_model = {}

models_to_tune = ["GIN", "GCN", "GAT"]

for model_name in models_to_tune:
    best_accuracy = 0
    best_params = {}

    for lr, hidden_channels, dropout_rate in itertools.product(learning_rates, hidden_channels_list, dropout_rates):
        heads = 8 if model_name == "GAT" else 1
        print(f"\nTuning {model_name} with Focal Loss, lr={lr}, hidden_channels={hidden_channels}, dropout={dropout_rate}")

        model, val_loss, val_accuracy = train_and_fine_tune_model_focal_loss(model_name, data, hidden_channels, lr, dropout_rate, num_epochs, heads)

        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_params = {
                'learning_rate': lr,
                'hidden_channels': hidden_channels,
                'dropout_rate': dropout_rate,
                'heads': heads
            }

    best_params_per_model[model_name] = best_params
    print(f"Best parameters for {model_name} with Focal Loss: {best_params}")
    print(f"Best validation accuracy for {model_name}: {best_accuracy}")



Tuning GIN with Focal Loss, lr=0.0001, hidden_channels=16, dropout=0.3
Epoch 0, Validation Loss: 0.34935997888704684, Validation Accuracy: 0.02547411149798963
Epoch 1, Validation Loss: 0.349365828794522, Validation Accuracy: 0.025470028471028805
Epoch 2, Validation Loss: 0.3493684997629587, Validation Accuracy: 0.02543138170418221
Epoch 3, Validation Loss: 0.34936535388009426, Validation Accuracy: 0.02543833550491941
Early stopping due to no improvement.

Tuning GIN with Focal Loss, lr=0.0001, hidden_channels=16, dropout=0.5
Epoch 0, Validation Loss: 0.3537443814147495, Validation Accuracy: 0.6135799302277858
Epoch 1, Validation Loss: 0.3537064173972932, Validation Accuracy: 0.6115038914746573
Epoch 2, Validation Loss: 0.35364883834907496, Validation Accuracy: 0.6085947258937022
Epoch 3, Validation Loss: 0.35365986328562793, Validation Accuracy: 0.6081083854022387
Epoch 4, Validation Loss: 0.3536846327870419, Validation Accuracy: 0.6095213172626469
Epoch 5, Validation Loss: 0.35360937

In [46]:
#evaluate
def evaluate_best_model(model_name, data, best_params):
    model_class = {
        "GIN": GIN,
        "GCN": GCN,
        "GAT": GAT
    }.get(model_name)

    if model_name == "GAT":
        model = model_class(in_channels=data.num_features, hidden_channels=best_params['hidden_channels'], out_channels=1, heads=best_params['heads']).to(device)
    else:
        model = model_class(in_channels=data.num_features, hidden_channels=best_params['hidden_channels'], out_channels=1).to(device)

    optimizer = torch.optim.SGD(model.parameters(), lr=best_params['learning_rate'])
    criterion = FocalLoss(gamma=1, weight=torch.tensor([1.0, 2.0]).to(device))

    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for test_data in test_loader:
            test_data.to(device)
            pred = model(test_data.x, test_data.edge_index, test_data.edge_attr)
            ground_truth = test_data.y

            pred_labels = (pred > 0.5).float()

            loss = criterion(pred, ground_truth.unsqueeze(1))
            test_loss += float(loss)

            correct += (pred_labels == ground_truth.unsqueeze(1)).sum().item()
            total += len(ground_truth)

            all_preds.extend(pred_labels.cpu().numpy())
            all_labels.extend(ground_truth.cpu().numpy())

    test_accuracy = correct / total
    test_loss /= len(test_loader)

    f1 = f1_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    precision = precision_score(all_labels, all_preds, average='binary')

    print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")
    print(f"Precision: {precision:.4f}, F1 Score: {f1:.4f}, Recall: {recall:.4f}")

In [47]:
#evaluate the models with the best parameters trained with Focal Loss
for model_name in models_to_tune:
    print(f"\nEvaluating {model_name} model with Focal Loss and the best hyperparameters...")
    best_params = best_params_per_model[model_name]
    evaluate_best_model(model_name, data, best_params)



Evaluating GIN model with Focal Loss and the best hyperparameters...
Test Loss: 0.35435650921045403, Test Accuracy: 0.8337709734549512
Precision: 0.0221, F1 Score: 0.0379, Recall: 0.1325

Evaluating GCN model with Focal Loss and the best hyperparameters...
Test Loss: 0.35336246975598207, Test Accuracy: 0.8409208431249072
Precision: 0.0227, F1 Score: 0.0387, Recall: 0.1298

Evaluating GAT model with Focal Loss and the best hyperparameters...
Test Loss: 0.3543073512040652, Test Accuracy: 0.9014466939473273
Precision: 0.0257, F1 Score: 0.0390, Recall: 0.0812


In [43]:
####### SMOTE FOCAL LOSS ########

In [23]:
#training and hyperparameter tuning with SMOTE and FocalLoss
def train_and_fine_tune_model_focal_loss_smote(model_name, data_smote, hidden_channels, learning_rate, dropout_rate, num_epochs=20, heads=8):
    model_class = {
        "GIN": GIN,
        "GCN": GCN,
        "GAT": GAT
    }.get(model_name)

    if model_name == "GAT":
        model = model_class(in_channels=data_smote.num_features, hidden_channels=hidden_channels, out_channels=1, heads=heads).to(device)
    else:
        model = model_class(in_channels=data_smote.num_features, hidden_channels=hidden_channels, out_channels=1).to(device)

    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    #FocalLoss with gamma = 1 and class weights
    criterion = FocalLoss(gamma=1, weight=torch.tensor([1.0, 2.0]).to(device))

    patience = 3
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch_data in train_loader_smote:  #SMOTE-applied training data loader
            optimizer.zero_grad()
            batch_data.to(device)
            pred = model(batch_data.x, batch_data.edge_index, batch_data.edge_attr)
            ground_truth = batch_data.y

            loss = criterion(pred, ground_truth.unsqueeze(1))
            loss.backward()
            optimizer.step()
            total_loss += float(loss)

        #no SMOTE on validation data
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for test_data in val_loader:
                test_data.to(device)
                pred = model(test_data.x, test_data.edge_index, test_data.edge_attr)
                ground_truth = test_data.y

                loss = criterion(pred, ground_truth.unsqueeze(1))
                val_loss += float(loss)
                correct += (pred.round() == ground_truth.unsqueeze(1)).sum().item()
                total += len(ground_truth)

        val_loss /= len(val_loader)
        val_accuracy = correct / total

        print(f"Epoch {epoch}, Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print("Early stopping due to no improvement.")
            break

    return model, best_val_loss, val_accuracy

In [28]:
#hyperparameter ranges
learning_rates = [0.0001, 0.001, 0.01]
hidden_channels_list = [16, 32, 64]
dropout_rates = [0.3, 0.5, 0.6]
num_epochs = 20

#best parameters and accuracy for each model
best_params_per_model_smote = {}

models_to_tune = ["GIN", "GCN", "GAT"]

for model_name in models_to_tune:
    best_accuracy = 0
    best_params = {}

    for lr, hidden_channels, dropout_rate in itertools.product(learning_rates, hidden_channels_list, dropout_rates):
        heads = 8 if model_name == "GAT" else 1
        print(f"\nTuning {model_name} with SMOTE and Focal Loss, lr={lr}, hidden_channels={hidden_channels}, dropout={dropout_rate}")

        model, val_loss, val_accuracy = train_and_fine_tune_model_focal_loss_smote(model_name, data_smote, hidden_channels, lr, dropout_rate, num_epochs, heads)

        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_params = {
                'learning_rate': lr,
                'hidden_channels': hidden_channels,
                'dropout_rate': dropout_rate,
                'heads': heads
            }

    best_params_per_model_smote[model_name] = best_params
    print(f"Best parameters for {model_name} with SMOTE and Focal Loss: {best_params}")
    print(f"Best validation accuracy for {model_name}: {best_accuracy}")


Tuning GIN with SMOTE and Focal Loss, lr=0.0001, hidden_channels=16, dropout=0.3
Epoch 0, Validation Loss: 0.35059627158470247, Validation Accuracy: 0.044641689341851164
Epoch 1, Validation Loss: 0.35041196500101396, Validation Accuracy: 0.04231117448967291
Epoch 2, Validation Loss: 0.35030235279878374, Validation Accuracy: 0.040305855296255605
Epoch 3, Validation Loss: 0.35017685662428144, Validation Accuracy: 0.039829569357370795
Epoch 4, Validation Loss: 0.35010089575504844, Validation Accuracy: 0.03862729203648789
Epoch 5, Validation Loss: 0.35002044405298255, Validation Accuracy: 0.037838600451467266
Epoch 6, Validation Loss: 0.34996951617319, Validation Accuracy: 0.036967505321740916
Epoch 7, Validation Loss: 0.3499099678792078, Validation Accuracy: 0.036445381968235394
Epoch 8, Validation Loss: 0.3498700958475581, Validation Accuracy: 0.03592172510443405
Epoch 9, Validation Loss: 0.34985287838774937, Validation Accuracy: 0.03564382851762823
Epoch 10, Validation Loss: 0.34980855

In [29]:
def evaluate_best_model(model_name, data, best_params):
    model_class = {
        "GIN": GIN,
        "GCN": GCN,
        "GAT": GAT
    }.get(model_name)

    if model_name == "GAT":
        model = model_class(in_channels=data.num_features, hidden_channels=best_params['hidden_channels'],
                            out_channels=1, heads=best_params['heads']).to(device)
    else:
        model = model_class(in_channels=data.num_features, hidden_channels=best_params['hidden_channels'],
                            out_channels=1).to(device)

    optimizer = torch.optim.SGD(model.parameters(), lr=best_params['learning_rate'])
    criterion = FocalLoss(gamma=1, weight=torch.tensor([1.0, 2.0]).to(device))

    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for test_data in test_loader:  #original test loader
            test_data.to(device)
            pred = model(test_data.x, test_data.edge_index, test_data.edge_attr)
            ground_truth = test_data.y

            pred_labels = (pred > 0.5).float()

            loss = criterion(pred, ground_truth.unsqueeze(1))
            test_loss += float(loss)

            correct += (pred_labels == ground_truth.unsqueeze(1)).sum().item()
            total += len(ground_truth)

            all_preds.extend(pred_labels.cpu().numpy())
            all_labels.extend(ground_truth.cpu().numpy())

    test_accuracy = correct / total
    test_loss /= len(test_loader)

    #classification metrics
    f1 = f1_score(all_labels, all_preds, average='binary')
    recall = recall_score(all_labels, all_preds, average='binary')
    precision = precision_score(all_labels, all_preds, average='binary')

    print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")
    print(f"Precision: {precision:.4f}, F1 Score: {f1:.4f}, Recall: {recall:.4f}")

In [30]:
#eevaluate the models with SMOTE and the best parameters trained with Focal Loss
for model_name in models_to_tune:
    print(f"\nEvaluating {model_name} model with SMOTE and Focal Loss and the best hyperparameters...")
    best_params = best_params_per_model_smote[model_name]
    evaluate_best_model(model_name, data, best_params)  # Still evaluating on original test data



Evaluating GIN model with SMOTE and Focal Loss and the best hyperparameters...
Test Loss: 0.35546025071783044, Test Accuracy: 0.138072123294857
Precision: 0.0248, F1 Score: 0.0482, Recall: 0.8850

Evaluating GCN model with SMOTE and Focal Loss and the best hyperparameters...
Test Loss: 0.3561051491620227, Test Accuracy: 0.8537345301307226
Precision: 0.0145, F1 Score: 0.0242, Recall: 0.0734

Evaluating GAT model with SMOTE and Focal Loss and the best hyperparameters...
Test Loss: 0.3508386579399961, Test Accuracy: 0.1764636639771945
Precision: 0.0231, F1 Score: 0.0449, Recall: 0.7851
